In [1]:
from dotenv import load_dotenv
import os
import requests
# Load environment variables from .env file
load_dotenv()

# Now you can access the environment variables
langchain_tracing_v2 = os.getenv('LANGCHAIN_TRACING_V2')
langchain_endpoint = os.getenv('LANGCHAIN_ENDPOINT')
langchain_api_key = os.getenv('LANGCHAIN_API_KEY')
openai_api_key = os.getenv('OPENAI_API_KEY')
# local_lm_server_url = os.getenv('LOCAL_LM_SERVER_URL')
# local_lm_server_url='http://localhost:1234/v1/chat/completions'
print(langchain_tracing_v2, langchain_endpoint, langchain_api_key, openai_api_key)

#openai_api_key

true https://api.smith.langchain.com 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 sk-proj-RcjMIRHFnwYgixH3gMuqT3BlbkFJCdycQXGoimYkkRExVZNP


In [8]:
import bs4
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.embeddings.spacy_embeddings import SpacyEmbeddings
from langchain_openai import OpenAI
from langchain_community.document_loaders import PyPDFLoader


In [5]:

client = OpenAI(openai_api_base="http://localhost:1234/v1", api_key="lm-studio", model="TheBloke/Mistral-7B-Instruct-v0.1-GGUF")


In [10]:
#### INDEXING ####
#CHANGE THE URL TO THE ONE YOU WANT TO INDEX

# Load Documents
# loader = WebBaseLoader(
#     web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
#     bs_kwargs=dict(
#         parse_only=bs4.SoupStrainer(
#             class_=("post-content", "post-title", "post-header")
#         )
#     ),
# )
# docs = loader.load()

pdfloader = PyPDFLoader("/Users/shauryatiwari/Desktop/Updated Resumes/Shaurya_Tiwari_RESUME.pdf")
pages = pdfloader.load_and_split()


print("initiate")


# Split
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(pages)

embedder = SpacyEmbeddings(model_name="en_core_web_sm")

# Embed
vectorstore = Chroma.from_documents(documents=splits, 
                                    embedding=embedder)

retriever = vectorstore.as_retriever()

#### RETRIEVAL and GENERATION ####

# Prompt
prompt = hub.pull("rlm/rag-prompt")

# Post-processing
def format_docs(pages):
    return "\n\n".join(doc.page_content for doc in pages)

# Chain
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | client
    | StrOutputParser()
)

# Question
rag_chain.invoke("Who is shaurya tiwari?")

initiate


' Shaurya Tiwari is an Indian actor and television personality.'